In [3]:
# Import Necessary Libraries and Packages
import sys  
import csv
import spacy
from time import process_time

nlp = spacy.load("en_core_web_lg")

# Set global variables
debug = False

In [54]:
debug = True

In [75]:

if debug:
    start_time = process_time()

# Set the appropriate input file
# movie_plots_import_file = "wiki_movie_plots_sample.csv" # 3248 Movies
movie_plots_import_file = "wiki_movie_plots_deduped.csv" #34,892 Movies


# Setup the plots array
plots = []

# Open import file and export all plots and titles into plots[]
with open(movie_plots_import_file, 'r', encoding='utf-8') as movie_plot_csv_file:
    reader = csv.reader(movie_plot_csv_file)
    next(reader, None)  # Skip the csv header row
    count=0

    for row in reader:
        plots.append((row[1],nlp(row[7])))
        count = count + 1
        if(count % 500 == 0):
            print(count, " movies processed")
            
print("Processing Done")

if debug:
    end_time = process_time()
    print("Time to process ", count, " records: ", end_time - start_time)
    print("Average processing timer per record: ", (end_time- start_time)/count)

# Print example when finished
print(plots[100])


500  movies processed
1000  movies processed
1500  movies processed
2000  movies processed
2500  movies processed
3000  movies processed
3500  movies processed
4000  movies processed
4500  movies processed
5000  movies processed
5500  movies processed
6000  movies processed
6500  movies processed
7000  movies processed
7500  movies processed
8000  movies processed
8500  movies processed
9000  movies processed
9500  movies processed
10000  movies processed
10500  movies processed
11000  movies processed
11500  movies processed
12000  movies processed
12500  movies processed
13000  movies processed
13500  movies processed
14000  movies processed
14500  movies processed
15000  movies processed
15500  movies processed
16000  movies processed
16500  movies processed
17000  movies processed
17500  movies processed
18000  movies processed
18500  movies processed
19000  movies processed
19500  movies processed
20000  movies processed
20500  movies processed
21000  movies processed
21500  movie

In [33]:
# Define generic similarity function
def simularity_printer(firstId, secondId):
    if debug:
        start_time = process_time()
    print("Similarity of ", plots[firstId][0], " and ", plots[secondId][0], ":", plots[firstId][1].similarity(plots[secondId][1]))
    if debug:
        end_time = process_time()
        print("Simularity calculation runtime ", end_time - start_time)

# Define generic 'best match' function
def best_match(id):
    if debug:
        start_time = process_time()
    best_score = 0
    best_match = "None"
    for plot in plots:
        if plot == plots[id]:
            continue
        score = plots[id][1].similarity(plot[1])
        if score > best_score:
            best_score = score
            best_match = plot[0]
    print("Best match for ",plots[id][0]," was: ",best_match," with score of ", best_score)
    if debug:
        end_time = process_time()
        print("Time to find best match from ", len(plots), " films: ", end_time - start_time)
        print("Average search time per film: ", (end_time-start_time)/len(plots))

In [21]:
# Print some tests/examples
simularity_printer(10, 250)
simularity_printer(10, 300)
simularity_printer(27, 72)
simularity_printer(27, 207)
simularity_printer(27, 270)

best_match(2000)
best_match(2500)
best_match(3000)

Similarity of  Dream of a Rarebit Fiend  and  Which Woman? : 0.9371971157790536
Simularity calculation runtime  0.0
Similarity of  Dream of a Rarebit Fiend  and  The Turn in the Road : 0.9343359550730539
Simularity calculation runtime  0.0
Similarity of  A Christmas Carol  and  Dough and Dynamite : 0.9311458919462225
Simularity calculation runtime  0.0
Similarity of  A Christmas Carol  and  Face Value : 0.9541526327456455
Simularity calculation runtime  0.0
Similarity of  A Christmas Carol  and  The Girl Who Stayed at Home : 0.9412193928939658
Simularity calculation runtime  0.0
Best match for  Top Hat  was:  The Awful Truth  with score of  0.9830821187041965
Time to find best match from  34886  films:  0.28125
Average search time per film:  8.061973284412086e-06
Best match for  At the Circus  was:  Gold Diggers in Paris  with score of  0.9813152325737858
Time to find best match from  34886  films:  0.234375
Average search time per film:  6.718311070343404e-06
Best match for  The Big S

In [34]:
best_match(2915)

Best match for  Rangers of Fortune  was:  Birth of a Nation  with score of  0.9835323053236005
Time to find best match from  3248  films:  0.013448000000011007
Average search time per film:  4.1403940886733395e-06


In [90]:
def top10_match(id):
    if debug:
        start_time = process_time()
    matches = [['BASE VALUE', 0]] # Schema: [ [title, score], [title, score]... ]
    
    for plot in plots:
        if plot == plots[id]:
            continue

        score = plots[id][1].similarity(plot[1]) # Get Similarity between current movie and match movie

        matches.append([plot[0], score]) # Add movie title and score to end of list
        matches = sorted(matches, key=lambda x: x[1], reverse=True) # Sort list by scores
        if len(matches) > 10: # Remove lowest score if length > 10
            matches.pop()
    
    end_time = process_time()
    # Print Results
    print("The best matches for", plots[id][0], "are:")
    i = 1
    for match in matches:
        print(str(i) + ".", match[0], "\t", match[1])
        i+=1

    print("\nTime to find best match from ", len(plots), " films: ", end_time - start_time)
    print("Average search time per film: ", (end_time-start_time)/len(plots))

top10_match(3331)

The best matches for In This Our Life are:
1. Diary of a Mad Black Woman 	 0.9897136333215154
2. Night World 	 0.9886594391290583
3. The Young Philadelphians 	 0.9881840178885334
4. Peyton Place 	 0.9879365684518179
5. Phone Call from a Stranger 	 0.9878915190776927
6. Ten North Frederick 	 0.9875224759181812
7. Devotion 	 0.9872186071767584
8. St. Elmo's Fire 	 0.9871606921712692
9. Red-Headed Woman 	 0.9871487847600823
10. Chained 	 0.9871140171446381

Time to find best match from  34886  films:  0.111587999999756
Average search time per film:  3.1986470217209194e-06


In [79]:
# Add column to Plots of plot without stopwords and punctuation - plots[id][2]
cleaned_plots = []
counter = 0

for plot in plots:
    if debug:
        counter+=1
        if (counter % 100 == 0):
            print(counter, "movie plots cleaned")
    token_list = []
    for token in plot[1]:
        if not nlp.vocab[token.text].is_stop and not nlp.vocab[token.text].is_punct and not token.text == "\n":
            token_list.append(token.text)

    cleaned_plots.append([plot[0], nlp(' '.join(str(token) for token in token_list))])

print(cleaned_plots[100][1])

100 movie plots cleaned
200 movie plots cleaned
300 movie plots cleaned
400 movie plots cleaned
500 movie plots cleaned
600 movie plots cleaned
700 movie plots cleaned
800 movie plots cleaned
900 movie plots cleaned
1000 movie plots cleaned
1100 movie plots cleaned
1200 movie plots cleaned
1300 movie plots cleaned
1400 movie plots cleaned
1500 movie plots cleaned
1600 movie plots cleaned
1700 movie plots cleaned
1800 movie plots cleaned
1900 movie plots cleaned
2000 movie plots cleaned
2100 movie plots cleaned
2200 movie plots cleaned
2300 movie plots cleaned
2400 movie plots cleaned
2500 movie plots cleaned
2600 movie plots cleaned
2700 movie plots cleaned
2800 movie plots cleaned
2900 movie plots cleaned
3000 movie plots cleaned
3100 movie plots cleaned
3200 movie plots cleaned
3300 movie plots cleaned
3400 movie plots cleaned
3500 movie plots cleaned
3600 movie plots cleaned
3700 movie plots cleaned
3800 movie plots cleaned
3900 movie plots cleaned
4000 movie plots cleaned
4100 movi

In [89]:
def top10_match_cleaned_plots(id):
    if debug:
        start_time = process_time()
    matches = [['BASE VALUE', 0]] # Schema: [ [title, score], [title, score]... ]
    
    for plot in cleaned_plots:
        if plot == cleaned_plots[id]:
            continue

        score = cleaned_plots[id][1].similarity(plot[1]) # Get Similarity between current movie and match movie

        matches.append([plot[0], score]) # Add movie title and score to end of list
        matches = sorted(matches, key=lambda x: x[1], reverse=True) # Sort list by scores
        if len(matches) > 10: # Remove lowest score if length > 10
            matches.pop()
    
    end_time = process_time()
    # Print Results
    print("The best matches for", cleaned_plots[id][0], "are:")
    i = 1
    for match in matches:
        print(str(i) + ".", match[0], "\t", match[1])
        i+=1

    print("\nTime to find best match from ", len(cleaned_plots), " films: ", end_time - start_time)
    print("Average search time per film: ", (end_time-start_time)/len(cleaned_plots))

top10_match_cleaned_plots(3331)

The best matches for In This Our Life are:
1. Diary of a Mad Black Woman 	 0.9636018318161498
2. The Young Philadelphians 	 0.9598892029942427
3. Devotion 	 0.9592136819588972
4. Phone Call from a Stranger 	 0.9578264969879424
5. The Man in the Net 	 0.9569263445181613
6. Matchstick Men 	 0.9566456674555075
7. The Last Time I Saw Paris 	 0.9563847880225985
8. The Ref 	 0.955756353615178
9.  The Family That Preys 	 0.9556927875827887
10. Tell It to the Judge 	 0.9555039502925088

Time to find best match from  34886  films:  0.10794999999961874
Average search time per film:  3.094364501508305e-06


In [6]:

# Take Plots[10] and Plots[100] and remove the stopwords
# TODO: Find more efficient way to remove stopwords
# TODO: Remove stopwords from ALL plots

# Create blank token list
first_token_list = []
# Tokenize plots - add each word to list
for token in plots[10][1]:
    first_token_list.append(token.text)

# Create a new string combining all the tokens (words) that are not stopwords
first_filtered_plot_str = ' '.join([str(token) for token in first_token_list if not nlp.vocab[token].is_stop])

# Repeat stopword removal for second plot
second_token_list = []
for token in plots[100][1]:
    second_token_list.append(token.text)      

second_filtered_plot_str = ' '.join([str(token) for token in second_token_list if not nlp.vocab[token].is_stop])

# Print standard simularity with stopwords
simularity_printer(10, 100)

# Print Filtered Stopword similarity to test stopword removal impact
print("Filtered Stopwords: ", nlp(first_filtered_plot_str).similarity(nlp(second_filtered_plot_str)))



Similarity of  Dream of a Rarebit Fiend  and  Should a Woman Divorce? : 0.8760381240648998
Filtered Stopwords:  0.7202336502162601


In [49]:
# Experiments with bag of words for keyword group mining
from bitarray import bitarray
from apyori import apriori


lexicon = {}
for plot in plots:
    for token in plot[1]:
        if not nlp.vocab[token.text].is_stop and not nlp.vocab[token.text].is_punct and not token.text == "\n":
            if token.lemma_ not in lexicon.keys():
                lexicon[token.lemma_] = 1
            else:
                lexicon[token.lemma_] += 1

print(list(lexicon)[0:25])
print(len(lexicon))

plot_word_bags = []
plot_word_lists = []

for plot in plots:
    plot_words = set()
    plot_words_list = []
    for token in plot[1]:
        if not nlp.vocab[token.text].is_stop and not nlp.vocab[token.text].is_punct:
            plot_words.add(token.lemma_)
    word_bag = bitarray()
    for word in lexicon:
        if word in plot_words:
            word_bag.append(1)
            plot_words_list.append(word)
        else:
            word_bag.append(0)
    plot_word_bags.append(word_bag)
    plot_word_lists.append(plot_words_list)

plot_num = 400
#print(plots[plot_num][0])
#print(plot_word_bags[plot_num])
#for i in range(0, len(lexicon)):
#    if plot_word_bags[plot_num][i]:
#        print(list(lexicon.keys())[i])

#Ignore compact bit vectors for now, see what happens with an out of the box apriori algorithm
results = list(apriori(plot_word_lists))
print("Apriori Test")
print("Total results: ", len(results))
for result in results:
    if(len(result.items) >= 3):
        print(result.items)

['bartender', 'work', 'saloon', 'serve', 'drink', 'customer', 'fill', 'stereotypically', 'irish', 'man', 'bucket', 'beer', 'Carrie', 'Nation', 'follower', 'burst', 'inside', 'assault', 'pull', 'hat', 'eye', 'dump', 'head', 'group', 'begin']
28380
Apriori Test
Total results:  487


TypeError: '>=' not supported between instances of 'frozenset' and 'int'